In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 795.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [2]:
!pip install pyngrok

In [3]:
! pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [4]:
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [26]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
f = open('/content/gemini.txt')
GOOGLE_API_KEY = f.read()

In [28]:
chat_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model="gemini-2.0-flash-exp", temperature=1)

In [29]:
# Create a connection with the database and
# return the chat message history for a session id

from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_message_history_from_db(session_id):
    chat_message_history = SQLChatMessageHistory(
                                   session_id=session_id,
                                   connection="sqlite:////content/sqlite.db"
                               )
    return chat_message_history

In [30]:
# Create a chat template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate(
                messages=[
                    ("system", """You are know to be polite and helpful AI bot.
                                        Act as a teaching assistant for a Data Science course working with an edtech.
                                        Your job here is to help students resolve there doubts regarding specific data science topics.
                                        If the doubt is not relevant to data science you can politely ask the user another doubt.
                                        """),
                    MessagesPlaceholder(variable_name="history"),
                    ("human", "{human_input}")
                ]
)

In [31]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [32]:
# Defining the chain
chain = chat_template | chat_model | output_parser

In [33]:
# Use RunnableWithMessageHistory to load
from langchain_core.runnables.history import RunnableWithMessageHistory

conversation_chain = RunnableWithMessageHistory(
                        chain,
                        get_session_message_history_from_db,
                        input_messages_key="human_input",
                        history_messages_key="history"
                    )

In [22]:
user_id = "Yohan"
config = {"configurable": {"session_id": user_id}}

input_prompt = {"human_input": "My name is Yohan. Can you Explain me the concept of KNN"}
response = conversation_chain.invoke(input_prompt, config=config)

response

"Hello Yohan, I am happy to help you with your doubt. KNN (K-Nearest Neighbors) is a simple and intuitive algorithm used for both classification and regression tasks in machine learning. Here's a breakdown of the concept:\n\n**Core Idea**\nKNN assumes that similar data points are located close to each other. So, to predict the class or value of a new data point, KNN looks at the 'K' nearest data points in the training set and makes a prediction based on their classes or values.\n\n**How it Works**\n\n1.  **Choose K:** The first step is to decide on the value of 'K'. This determines how many neighbors will influence the classification.\n2.  **Distance Calculation:** When you have a new data point that you want to classify, KNN calculates the distance between this point and all other points in the training data. Common distance metrics include:\n\n    *   **Euclidean Distance:** Straight line distance between two points.\n    *   **Manhattan Distance:** Sum of the absolute differences of

In [37]:
def chat_bot(session_id, prompt):
    config = {"configurable": {"session_id": user_id}}
    input_prompt = {"human_input": prompt}

    response = conversation_chain.invoke(input_prompt, config=config)

    return response

In [38]:
user_id = "Yohan"
input_prompt = "Do you remember my name?"
chat_bot(session_id=user_id, prompt=input_prompt)

"Yes, Yohan, I remember your name.\n\nIs there something else I can help you with regarding Data Science? Perhaps you'd like to explore another algorithm, discuss evaluation metrics, or anything else related to the field? Let me know!"

In [ ]:
chat_bot(session_id, prompt)

In [39]:
%%writefile app.py
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory

# Load API Key
with open("gemini.txt", "r") as f:
    GOOGLE_API_KEY = f.read().strip()

# Initialize Chat Model
chat_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model="gemini-1.5-pro", temperature=1)

# Function to retrieve chat history from SQLite database
def get_session_message_history_from_db(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection="sqlite:///sqlite.db")

# Chat Prompt Template
chat_template = ChatPromptTemplate(
    messages=[
        ("system", """You are a polite and helpful AI teaching assistant for a Data Science course.
                      Your job is to help students resolve their doubts regarding specific data science topics.
                      If the question is not relevant to data science, politely ask the user for another question.
                   """),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}")
    ]
)

# Output Parser
output_parser = StrOutputParser()

# Define the Conversation Chain
conversation_chain = RunnableWithMessageHistory(
    chat_template | chat_model | output_parser,
    get_session_message_history_from_db,
    input_messages_key="human_input",
    history_messages_key="history"
)

# Streamlit UI
st.title("🧠 Conversational AI Data Science Tutor")
st.write("Ask any Data Science-related question, and I'll assist you!")

# Initialize chat history in session state if not present
if "session_id" not in st.session_state:
    st.session_state.session_id = "user_1"  # Unique session ID for chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display previous chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# User Input
user_input = st.chat_input("Type your data science question...")
if user_input:
    # Display user message
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # Get AI response
    config = {"configurable": {"session_id": st.session_state.session_id}}
    response = conversation_chain.invoke({"human_input": user_input}, config=config)

    # Display AI response
    st.session_state.messages.append({"role": "assistant", "content": response})
    with st.chat_message("assistant"):
        st.markdown(response)


Writing app.py


In [42]:
!wget -q -O - ipv4.icanhazip.com

35.202.107.87


In [43]:
!streamlit run /content/app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.202.107.87:8501

⠴⠦⠧your url is: https://gold-friends-brush.loca.lt
  Stopping...
E0000 00:00:1741587528.606739   15293 init.cc:232] grpc_wait_for_shutdown_with_timeout() timed out.
^C
